In [29]:
lis1 = b'v\x00\x00a'
print ("The iterable conversion results in : "  + str(bytes(lis1)))
print(int.from_bytes(lis1[:-1], "little", signed = True))


The iterable conversion results in : b'v\x00\x00a'
118


In [30]:
import serial

with serial.Serial('/dev/tty.usbmodem2101', baudrate=9600) as ser:
    for i in range(20):
        s = ser.read(4)        # read up to ten bytes (timeout)
        print(int.from_bytes(s[:-1], "little"))

14
14
14
14
14
15
15
14
14
14
14
14
14
14
14
14
14
15
14
15


In [21]:
import sys
import glob
import serial

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

if __name__ == '__main__':
    print(serial_ports())


['/dev/tty.Bluetooth-Incoming-Port', '/dev/tty.usbmodem2101']


In [13]:
import serial
import time
ser = serial.Serial(port = '/dev/tty.usbmodem2101', 
                           baudrate = 115200, 
                           timeout = 0)
i = 0
while(True):
    line = ser.readline()   # read a byte
    if line:
        string = line.decode()  # convert the byte string to a unicode string
        num = int(string) # convert the unicode string to an int
        print(num)
    if(i > 100):
        break
    i+=1;


In [31]:
# import serial
# import time
# serialPort = serial.Serial(port = '/dev/tty.usbmodem2101', 
#                            baudrate = 115200, 
#                            timeout = 0)
# while True:
#     input_word = input("input on/off : ").strip()
#     if (input_word == 'done'):
#         print("\nFINISHED\n")
#         break
#     serialPort.write(input_word.encode())   #We have to encode the word into bytes
#     time.sleep(0.5)
#     print(serialPort.readline().decode('ascii')) #Arduino sends bytes back so we have to encode it again
# serialPort.close()